# Описание данных:

Данные содержат информацию о показателях здоровья, собранных с умных часов.

## Структура данных:

| Название столбца            | Описание                                        |
|-----------------------------|-------------------------------------------------|
| **User ID**                 | Уникальный идентификатор пользователя           |
| **Heart Rate (BPM)**        | Частота сердечных сокращений (ударов в минуту)  |
| **Blood Oxygen Level (%)**  | Уровень насыщения крови кислородом              |
| **Step Count**              | Количество шагов за период                      |
| **Sleep Duration (hours)**  | Продолжительность сна в часах                   |
| **Activity Level**          | Уровень активности пользователя                 |
| **Stress Level**            | Уровень стресса (шкала от 1 до 10)              |